In [106]:
import dgl
import torch
from dgl.data.utils import save_graphs

(g1,), _ = dgl.load_graphs('graph1.dgl')
(g2,), _ = dgl.load_graphs('graph2.dgl')
(g3,), _ = dgl.load_graphs('graph3.dgl')


In [107]:
g = dgl.batch([g2,g3])

In [108]:
g

Graph(num_nodes=379, num_edges=408,
      ndata_schemes={'feat': Scheme(shape=(4,), dtype=torch.float32), 'CRS_DEP_TIME': Scheme(shape=(1,), dtype=torch.float32), 'DEP_DELAY': Scheme(shape=(1,), dtype=torch.float32), 'WHEELS_OFF': Scheme(shape=(1,), dtype=torch.float32), 'CRS_ARR_TIME': Scheme(shape=(1,), dtype=torch.float32), 'ARR_DELAY': Scheme(shape=(1,), dtype=torch.float32), 'DEST': Scheme(shape=(1,), dtype=torch.float32), 'ORIGIN': Scheme(shape=(1,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'AIRCRAFT_NUM': Scheme(shape=(1,), dtype=torch.float32), 'INTERVAL_TIME': Scheme(shape=(1,), dtype=torch.float32), 'AIRPORT': Scheme(shape=(1,), dtype=torch.float32)})

In [109]:
g.adj()

tensor(indices=tensor([[  0,   0,   0,   0,   0,   5,   6,   6,   6,   6,  10,
                          6,   6,   6,   6,   6,   6,  17,  17,  17,  17,   6,
                          6,   6,  24,  24,  24,  24,   6,   6,   5,  31,  31,
                          5,  34,  34,  34,  35,  35,  35,  34,  34,  34,  39,
                         39,  39,  34,   5,  42,  42,  44,  44,  44,  44,  44,
                          5,  50,  50,  50,  50,  50,  50,  50,  57,  57,  57,
                         50,  61,  61,  50,  64,  64,  64,  64,  64,  50,  50,
                         71,  71,  71,  71,  71,  71,  71,  78,  78,  71,  50,
                         50,   5,  84,  84,  86,  86,  86,  86,  86,  86,  86,
                         86,  86,  84,  91,  91,  91,  93,  91,  91,  91,  91,
                         91,  91,  91,  91,   5, 102, 102, 102,   5,   5,   5,
                          0,   0, 110, 111, 112, 111, 111, 111, 111, 110,   0,
                        119, 120, 120, 122, 122, 122

In [110]:
g.num_nodes()

379

In [111]:

    # 构建一个2层的GNN模型
    import dgl.nn as dglnn
    import torch.nn as nn
    import torch.nn.functional as F
    class SAGE(nn.Module):
        def __init__(self, in_feats, hid_feats, out_feats):
            super().__init__()
            # 实例化SAGEConve，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregator_type是聚合函数的类型
            self.conv1 = dglnn.SAGEConv(
                in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
            self.conv2 = dglnn.SAGEConv(
                in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')
      
        def forward(self, graph, inputs):
            # 输入是节点的特征
            h = self.conv1(graph, inputs)
            h = F.relu(h)
            h = self.conv2(graph, h)
            return h


In [119]:
import random
def getRandomIndex(n, x):
    # 索引范围为[0, n), 随机选x个不重复
    index = random.sample(range(n), x)
    return index

In [121]:
idx=getRandomIndex(1000,10)

[566, 726, 992, 921, 34, 681, 429, 790, 800, 234]

In [135]:
num=g.num_nodes()
print(num)
num1=int(num/10)
idx=getRandomIndex(num,9*num1)
train_mask=torch.zeros(num).bool()
print(train_mask)
test_mask=torch.zeros(num).bool()
valid_mask=torch.zeros(num).bool()
#valid_mask[0:num1]=True
#test_mask[num1:2*num1]=True
train_mask[idx]=True
test_mask=~train_mask
valid_mask=~train_mask
print(train_mask)
print(test_mask)

379
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, Fal

In [136]:
g.ndata['label']

tensor([1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,

In [137]:
#二分类
def one_hot(target):
	#target is a list
	#我们设第一个label为0标签，这个取决于你，你可以自己设定一个labelname
	temp=target[0]
	label=[]
	for i in range(len(target)):
		if target[i]==temp:
			label.append(0)
		else:
			label.append(1)
	return label



In [138]:
    #node_features = torch.cat((g.ndata['DEP_DELAY'],g.ndata['WHEELS_OFF']),1)
    node_features = g.ndata['feat']
    node_labels = g.ndata['label']
    n_features = node_features.shape[1]
    n_labels = int(node_labels.max().item() + 1)
    print(n_features)
    print(n_labels)

4
2


In [139]:
def evaluate(model, graph, features, labels, mask):
     model.eval()
     with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


In [172]:
    import matplotlib.pyplot as plt    
    l_n = 0
    a_n = 0
    loss_x = []
    loss_y = []
    acc_x = []
    acc_y = []
    
    
    
    
    
    model = SAGE(in_feats=n_features, hid_feats=75, out_feats=n_labels)
    opt = torch.optim.Adam(model.parameters())
    
    for epoch in range(1000):
        model.train()
        # 使用所有节点(全图)进行前向传播计算
        logits = model(g, node_features)
        # 计算损失值
        #loss = F.mse_loss(logits[train_mask], node_labels[train_mask])
        loss = F.cross_entropy(logits[train_mask],node_labels[train_mask])
        loss_test=F.cross_entropy(logits[test_mask],node_labels[test_mask])
        # 计算验证集的准确度
        acc = evaluate(model, g, node_features, node_labels, valid_mask)
        # 进行反向传播计算
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss.item())
        #print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
        print(loss_test.item())
        print(acc)
        # 如果需要的话，保存训练好的模型。本例中省略。



429.6805419921875
680.9608764648438
0.3695652173913043
388.984130859375
633.0449829101562
0.41304347826086957
349.5326843261719
587.3863525390625
0.43478260869565216
312.81378173828125
545.1123657226562
0.4782608695652174
280.00628662109375
505.5715026855469
0.5
250.53164672851562
467.927001953125
0.5217391304347826
224.20721435546875
432.22711181640625
0.5652173913043478
201.0260009765625
400.2319030761719
0.5869565217391305
180.32687377929688
369.4946594238281
0.5869565217391305
161.3136444091797
339.1615905761719
0.6086956521739131
144.91131591796875
311.3946533203125
0.6304347826086957
131.0114288330078
284.71319580078125
0.6521739130434783
120.13111114501953
260.2491455078125
0.6739130434782609
111.9820556640625
238.2607421875
0.6956521739130435
104.87775421142578
216.68841552734375
0.6956521739130435
98.65242767333984
195.84451293945312
0.7391304347826086
93.11788177490234
176.9543914794922
0.7391304347826086
87.81256866455078
159.0917510986328
0.782608695652174
82.92445373535156

20.763093948364258
0.8695652173913043
3.9984705448150635
20.65668296813965
0.8695652173913043
3.941030502319336
20.55707359313965
0.8695652173913043
3.88258695602417
20.494871139526367
0.8695652173913043
3.827951669692993
20.48537254333496
0.8695652173913043
3.7670300006866455
20.531415939331055
0.8695652173913043
3.7175240516662598
20.59148597717285
0.8695652173913043
3.6692044734954834
20.584022521972656
0.8695652173913043
3.6211066246032715
20.499061584472656
0.8695652173913043
3.568603992462158
20.400161743164062
0.8695652173913043
3.52028226852417
20.320924758911133
0.8695652173913043
3.466374158859253
20.286596298217773
0.8695652173913043
3.416656732559204
20.285959243774414
0.8695652173913043
3.3736259937286377
20.2589054107666
0.8695652173913043
3.328356981277466
20.187875747680664
0.8695652173913043
3.2853384017944336
20.105363845825195
0.8695652173913043
3.240501880645752
20.038339614868164
0.8695652173913043
3.1944122314453125
19.99989891052246
0.8695652173913043
3.152140378

17.511465072631836
0.8260869565217391
0.43798819184303284
17.52845001220703
0.8260869565217391
0.43545666337013245
17.547313690185547
0.8260869565217391
0.4328885078430176
17.567468643188477
0.8260869565217391
0.430308997631073
17.589235305786133
0.8260869565217391
0.42778280377388
17.611724853515625
0.8260869565217391
0.42527687549591064
17.629968643188477
0.8260869565217391
0.42288312315940857
17.646251678466797
0.8260869565217391
0.4204185903072357
17.661865234375
0.8260869565217391
0.41789937019348145
17.677797317504883
0.8260869565217391
0.4153578579425812
17.68824005126953
0.8260869565217391
0.4127977192401886
17.69515037536621
0.8260869565217391
0.41024714708328247
17.701452255249023
0.8260869565217391
0.4077368378639221
17.709331512451172
0.8260869565217391
0.40523040294647217
17.715307235717773
0.8260869565217391
0.40272319316864014
17.721349716186523
0.8260869565217391
0.4004969000816345
17.731090545654297
0.8260869565217391
0.39852002263069153
17.738035202026367
0.8260869565

0.14462874829769135
18.027841567993164
0.8260869565217391
0.14300377666950226
18.034046173095703
0.8260869565217391
0.1412612795829773
18.042400360107422
0.8260869565217391
0.13958033919334412
18.044652938842773
0.8260869565217391
0.1379413604736328
18.05588722229004
0.8260869565217391
0.13621318340301514
18.058408737182617
0.8260869565217391
0.13453954458236694
18.072154998779297
0.8260869565217391
0.1329168826341629
18.07488250732422
0.8260869565217391
0.13119803369045258
18.08537483215332
0.8260869565217391
0.12956972420215607
18.092803955078125
0.8260869565217391
0.12800586223602295
18.096572875976562
0.8260869565217391
0.12631098926067352
18.10918426513672
0.8260869565217391
0.12469300627708435
18.111364364624023
0.8260869565217391
0.12309607863426208
18.121265411376953
0.8260869565217391
0.12143775075674057
18.128829956054688
0.8260869565217391
0.11984504014253616
18.135726928710938
0.8260869565217391
0.11826064437627792
18.14366340637207
0.8260869565217391
0.11661273241043091
18

18.351272583007812
0.8260869565217391
0.0233603548258543
18.354310989379883
0.8260869565217391
0.02275729738175869
18.355627059936523
0.8260869565217391
0.02215716987848282
18.358552932739258
0.8260869565217391
0.021562252193689346
18.36104393005371
0.8260869565217391
0.020970823243260384
18.362361907958984
0.8260869565217391
0.020382186397910118
18.364761352539062
0.8260869565217391
0.01979868859052658
18.368328094482422
0.8260869565217391
0.019222114235162735
18.370691299438477
0.8260869565217391
0.01865282841026783
18.372438430786133
0.8260869565217391
0.018091747537255287
18.375417709350586
0.8260869565217391
0.017544502392411232
18.378034591674805
0.8260869565217391
0.017005495727062225
18.3785400390625
0.8260869565217391
0.016479037702083588
18.379405975341797
0.8260869565217391
0.015958769246935844
18.379724502563477
0.8260869565217391
0.01545154582709074
18.381011962890625
0.8260869565217391
0.014952768571674824
18.38191032409668
0.8260869565217391
0.014466643333435059
18.38353

18.243581771850586
0.8260869565217391
0.0017010909505188465
18.241880416870117
0.8260869565217391
0.0016893463907763362
18.240707397460938
0.8260869565217391
0.0016775489784777164
18.239721298217773
0.8260869565217391
0.0016655938234180212
18.238828659057617
0.8260869565217391
0.0016537554329261184
18.23823356628418
0.8260869565217391
0.0016420980682596564
18.23804473876953
0.8260869565217391
0.0016306535108014941
18.237886428833008
0.8260869565217391
0.0016196133801713586
18.237323760986328
0.8260869565217391
0.0016082682413980365
18.2359676361084
0.8260869565217391
0.0015973319532349706
18.234724044799805
0.8260869565217391
0.0015867408365011215
18.233688354492188
0.8260869565217391
0.0015762589173391461
18.232376098632812
0.8260869565217391
0.0015656438190490007
18.231332778930664
0.8260869565217391
0.0015553822740912437
18.230575561523438
0.8260869565217391
0.0015449794009327888
18.23011589050293
0.8260869565217391
0.001534910756163299
18.22970199584961
0.8260869565217391
0.0015247

18.15131950378418
0.8260869565217391
0.000803109840489924
18.150863647460938
0.8260869565217391
0.0008000541711226106
18.15049171447754
0.8260869565217391
0.0007970939041115344
18.150108337402344
0.8260869565217391
0.0007941816002130508
18.14968490600586
0.8260869565217391
0.0007911412976682186
18.14896583557129
0.8260869565217391
0.0007882366771809757
18.148313522338867
0.8260869565217391
0.0007853537681512535
18.14774513244629
0.8260869565217391
0.0007824168424122036
18.14726448059082
0.8260869565217391
0.0007797411526553333
18.14681053161621
0.8260869565217391
0.0007767195929773152
18.146146774291992
0.8260869565217391
0.0007739565335214138
18.145427703857422
0.8260869565217391
0.0007711907383054495
18.144847869873047
0.8260869565217391
0.0007684683660045266
18.144515991210938
0.8260869565217391
0.000765784818213433
18.144336700439453
0.8260869565217391
0.0007630711770616472
18.144163131713867
0.8260869565217391
0.0007602632394991815
18.143945693969727
0.8260869565217391
0.000757496

In [82]:
print(logits[train_mask].shape)
print(node_labels[train_mask].shape)

torch.Size([581, 2])
torch.Size([581])


In [101]:
t5=temp2.squeeze()
t5=t5.long()

In [102]:
print(t5.shape)

torch.Size([54])


In [141]:
probabilities = model.predict_proba(test_x)

AttributeError: 'SAGE' object has no attribute 'predict_proba'

In [151]:
logits = logits[test_mask]
print(logits)

labels = node_labels[test_mask]

print(labels)
_, indices = torch.max(logits, dim=1)


tensor([[-1035.7208, -1082.5298],
        [  182.8829,  -698.6850],
        [  907.2913,   924.0951],
        [-4923.3008, -2600.9971],
        [ 1051.8176,   427.5142],
        [ 1268.3265,  1258.6403],
        [  169.0658,  -344.0612],
        [   30.0942,  -438.6878],
        [ 1588.7330,   899.2083],
        [  551.5488,  -446.7591],
        [  424.6975,  -470.6208],
        [ -475.0356, -1053.7142],
        [  543.2178,   173.4458],
        [  104.8746,  -869.1998],
        [-1480.6757, -1001.8768],
        [  214.2089,   104.8120],
        [ -590.3461,  -584.2427],
        [  501.2588,   182.7095],
        [  974.9406,   224.2963],
        [ 1179.5753,   541.1630],
        [ 1354.7339,   858.9672],
        [  710.8356,   461.0884],
        [   84.0449,  -495.4872],
        [  545.4564,   261.0651],
        [  611.4240,   254.9522],
        [  612.6103,   285.1067],
        [  961.8168,   350.4724],
        [  708.0507,   297.2830],
        [  663.5809,   193.3387],
        [  156

In [152]:
indices

tensor([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [171]:
TP = torch.sum((v1)&(v2))
print(TP)
FP = torch.sum((v1)&(~v2))
print(FP)
FN = torch.sum((~v1)&(v2))
print(FN)
TN = torch.sum((~v1)&(~v2))
print(TN)



tensor(6)
tensor(3)
tensor(1)
tensor(36)


In [159]:
v1=indices == 1
v1

tensor([False, False,  True,  True, False, False, False, False, False, False,
        False, False, False, False,  True, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True,  True, False,  True,  True,  True, False,
        False, False, False, False, False, False])

In [160]:
v2=labels==1
v2

tensor([ True, False, False,  True, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True,  True, False,  True, False,  True, False,
        False, False, False, False, False, False])

In [164]:
v1&v2

tensor([False, False, False,  True, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True,  True, False,  True, False,  True, False,
        False, False, False, False, False, False])